## LLM Authentication Examples

This notebook demonstrates the different ways to authenticate and access LLMs with Spindle, including:

- **Auto-Detection:** Automatically detect and use GCP Vertex AI credentials when available
- **Explicit Vertex AI Configuration:** Manually configure Vertex AI with project ID and region
- **Mixed Authentication:** Use Vertex AI for some models, direct API keys for others
- **Direct API Keys:** Traditional approach using `ANTHROPIC_API_KEY` and `OPENAI_API_KEY`

### Key Features

- **Preference for Vertex AI:** When both GCP credentials and API keys are available, Vertex AI is preferred
- **Automatic Fallback:** If Vertex AI auth fails, automatically falls back to direct API keys
- **Support for Multiple Providers:** Anthropic (Claude), OpenAI (GPT), and Google (Gemini) models
- **Both SDK and API Approaches:** Supports Anthropic Vertex SDK and MAAS API methods


## Setup

First, let's import the necessary modules:


In [ ]:
import os  # 'os' imported but unused
import sys  # 'sys' imported but unused
from pprint import pprint

# Import Spindle components
from spindle import SpindleExtractor, create_ontology
from spindle.llm_config import (
    LLMConfig,
    detect_available_auth,
    setup_llm_config,  # 'spindle.llm_config.setup_llm_config' imported but unused
    get_anthropic_vertex_client,  # 'spindle.llm_config.get_anthropic_vertex_client' imported but unused
    create_vertex_maas_request,  # 'spindle.llm_config.create_vertex_maas_request' imported but unused
    AuthMethod,
    Provider,
)

print("✅ Imports successful")


## Section 1: Auto-Detection

The simplest approach - let Spindle automatically detect available credentials and choose the best authentication method.

### How Auto-Detection Works

1. Check for GCP Vertex AI credentials:
   - `GOOGLE_APPLICATION_CREDENTIALS` environment variable
   - Default GCP credentials (via `gcloud auth application-default login`)
   - `GCP_PROJECT_ID` and `GCP_REGION` environment variables

2. Check for direct API keys:
   - `ANTHROPIC_API_KEY`
   - `OPENAI_API_KEY`
   - `GOOGLE_API_KEY`

**Prefer Vertex AI:** If both GCP and API keys are available, Vertex AI is preferred

Let's see what credentials are detected:


In [ ]:
# Detect available authentication methods
config = detect_available_auth()

print("\n=== Detected Configuration ===")
pprint(config.to_dict())

print("\n=== Authentication Method Selection ===")
for provider in Provider.ANTHROPIC, Provider.OPENAI, Provider.GOOGLE:
    auth_method = config.get_auth_method_for_provider(provider)
    print(f"{provider.value:12} -> {auth_method.value if auth_method else 'Not available'}")


### Using Auto-Detection with SpindleExtractor

When you create a `SpindleExtractor` without specifying `llm_config`, it automatically detects credentials:


In [ ]:
# Define a simple ontology
entity_types = [
    {"name": "Person", "description": "A human being"},
    {"name": "Organization", "description": "A company or institution"},
    {"name": "Location", "description": "A place or geographical location"},
]

relation_types = [
    {"name": "works_at", "description": "Employment relationship", "domain": "Person", "range": "Organization"},
    {"name": "located_in", "description": "Location relationship", "domain": "Organization", "range": "Location"},
]

ontology = create_ontology(entity_types, relation_types)

# Create extractor with auto-detection (default behavior)
extractor = SpindleExtractor(ontology=ontology, auto_detect_auth=True)

print(f"Extractor configured with: {extractor.llm_config.preferred_auth_method.value if extractor.llm_config else 'No config'}")
print(f"Available methods: {[m.value for m in extractor.llm_config.available_auth_methods] if extractor.llm_config else []}")


In [ ]:
# Extract triples using auto-detected credentials
text = "Alice Johnson works at TechCorp, which is located in San Francisco."

try:
    result = extractor.extract(text, source_name="example_auto")
    print(f"✅ Extracted {len(result.triples)} triples")
    for triple in result.triples:
        print(f"- {(triple.subject.name)} -> {triple.predicate} -> {triple.object.name} ({triple.object.type})")
except Exception as exc:
    print(f"❌ Extraction failed: {exc}")
    print("⚠️ Note: This might be because no valid credentials were detected.")


## Section 2: Explicit Vertex AI Configuration

Sometimes you want to explicitly configure Vertex AI credentials, especially when:
- You have multiple GCP projects
- You want to use a specific region
- You want to override auto-detection

### Approach 1: Using Anthropic Vertex AI SDK

This is the recommended approach for Anthropic models via Vertex AI:


In [ ]:
# Explicit configuration for Vertex AI
vertex_config = LLMConfig(
    gcp_project_id="your-project-id",  # Replace with your project ID
    gcp_region="us-east5",  # Claude Sonnet 4 is available in us-east5
    preferred_auth_method=AuthMethod.VERTEX_AI,
)

print("=== Explicit Vertex AI Configuration ===")
print(f"Project ID: {vertex_config.gcp_project_id}")
print(f"Region: {vertex_config.gcp_region}")
print(f"Has Vertex AI auth: {vertex_config.has_vertex_ai_auth()}")


## Summary

This notebook demonstrated four approaches to LLM authentication with Spindle:

1. **Auto-Detection** (Recommended): Let Spindle detect and choose the best method
2. **Explicit Vertex AI**: Manually configure GCP project and region
3. **Mixed Authentication**: Use different methods for different providers
4. **Direct API Keys**: Traditional approach with API keys

### Quick Start Guide

```python
# Option 1: Auto-detection (easiest)
extractor = SpindleExtractor(ontology)
# That's it! Spindle will detect and use the best available auth method

# Option 2: Explicit Vertex AI
from spindle.llm_config import LLMConfig, AuthMethod
config = LLMConfig(
    gcp_project_id="your-project",
    gcp_region="us-east5",
    preferred_auth_method=AuthMethod.VERTEX_AI,
)
extractor = SpindleExtractor(ontology, llm_config=config)

# Option 3: Direct API keys only
Just set environment variables:
export ANTHROPIC_API_KEY=...
export OPENAI_API_KEY=...
extractor = SpindleExtractor(ontology)  # Will use API keys
```

### Next Steps

- See `docs/VERTEX_AI_SETUP.md` for detailed Vertex AI configuration
- See `docs/QUICKSTART.md` for general Spindle usage
- Check other example notebooks for specific use cases
